In [ ]:
import ccxt  # noqa: E402
import pandas as pd
import numpy as np
import json

from clients import BinanceClient

In [ ]:
binance = BinanceClient()
binance.fetch_balance()

symbol = 'BTC/USDT'
timeframe = '1h'
df = binance.get_ohlc(symbol, timeframe)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df['datetime'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

In [ ]:
# -*- coding: utf-8 -*-


binance = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'

ohlcv = binance.fetch_ohlcv(symbol, timeframe)
#[ timestamp, open, high, low, close, volume ]
df=pd.DataFrame(ohlcv).rename({0:'timestamp', 1:'open', 2:'high', 3:'low', 4:'close', 5:'volume'},axis=1)
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
df

In [ ]:
from web3 import Web3
import json
import os

from dotenv import load_dotenv
load_dotenv()

abi_path = './abi.json'
wss_url = os.getenv('wss_url')

router_address = "0xE592427A0AEce92De3Edee1F18E0157C05861564"  # Uniswap v3 SwapRouter

with open(abi_path) as f:
    abi = json.load(f)

interface = Web3().eth.contract(abi=abi)

In [ ]:
provider = Web3(Web3.WebsocketProvider(wss_url))

def handle_transaction(tx_hash):
    txn_data = provider.eth.get_transaction(tx_hash)
    if txn_data:
        gas = txn_data['gasPrice']
        if txn_data['to'] == router_address and "0x414bf389" in txn_data['input']:
            print("hash: ", tx_hash)
            decoded = interface.decode_function_input(
                "exactInputSingle((address,address,uint24,address,uint256,uint256,uint256,uint160))", txn_data['input'])
            log_txn(decoded, gas)

def log_txn(data, gas):
    print("tokenIn: ", data['params']['tokenIn'])
    print("tokenOut: ", data['params']['tokenOut'])
    print("amount: ", data['params']['amountOutMinimum'])
    print("gasPrice: ", gas)
    print("-------------")

handle_transaction()
# provider.eth.subscribe('pending_transactions', handle_transaction)

In [ ]:
# Set up web3 provider
w3 = Web3(Web3.WebsocketProvider(wss_url))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
router = "0xE592427A0AEce92De3Edee1F18E0157C05861564"
# Contract instance
contract = w3.eth.contract(address=router, abi=abi)

def main():
    # Event handler
    def handle_event(event):
        txn_data = w3.eth.getTransaction(event)
        if txn_data:
            gas = txn_data['gasPrice']
            if txn_data.to == router and "0x414bf389" in txn_data['input']:
                print("hash: ", txn_data['hash'])
                decoded = contract.decode_function_input("exactInputSingle((address,address,uint24,address,uint256,uint256,uint256,uint160))",  txn_data['input'])
                log_txn(decoded, gas)

    # Subscribe to pending transactions
    w3.eth.subscribe('pending_transactions', handle_event)

def log_txn(data, gas):
    print("tokenIn: ", data['params']['tokenIn'])
    print("tokenOut: ", data['params']['tokenOut'])
    print("amount: ", data['params']['amountOutMinimum'])
    print("gasPrice: ", gas)
    print("-------------")

if __name__ == "__main__":
    main()